In [30]:
import os
import pandas as pd
import logging
import datetime
import time

import ml_investing_wne.config as config
from ml_investing_wne.xtb.xAPIConnector import APIClient, APIStreamClient, loginCommand

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [31]:
start_date_dict = {60: datetime.datetime(2019, 1, 1, 1, 0, 0, 0), 1440: datetime.datetime(2006, 1, 1, 1, 0, 0, 0)}
symbols = ['US500', 'VIX', 'US100','US2000','US30','DE30','W20','VSTOXX','EU50','UK100',
           'OIL.WTI','GOLD','COPPER','TNOTE','BUND10Y','SCHATZ2Y',
          'EURCHF', 'EURGBP', 'EURUSD', 'USDCHF', 'USDJPY', 'EURPLN']
periods = [60, 1440]

In [32]:
start_date_dict

{60: datetime.datetime(2019, 1, 1, 1, 0),
 1440: datetime.datetime(2006, 1, 1, 1, 0)}

In [33]:
client = APIClient()
# connect to RR socket, login
loginResponse = client.execute(loginCommand(userId=config.userId, password=config.password))
logger.info(str(loginResponse))

[2022-09-25 10:33:38,841][connect:76] Socket connected
[2022-09-25 10:33:38,843][_waitingSend:90] Sent: b'{"command": "login", "arguments": {"userId": 1590700, "password": "Symelgitara7", "appName": ""}}'
[2022-09-25 10:33:38,944][_read:110] Received: {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-00004e4e-0002776f-e6de204905a5a91f-0eacb1e1'}
[2022-09-25 10:33:38,946][<module>:4] {'status': True, 'streamSessionId': 'ca005bfffe1f3af5-00004e4e-0002776f-e6de204905a5a91f-0eacb1e1'}


In [34]:
# check if user logged in correctly
if (loginResponse['status'] == False):
    print('Login failed. Error code: {0}'.format(loginResponse['errorCode']))

# get ssId from login response
ssid = loginResponse['streamSessionId']

In [35]:
for period in periods:
    for symbol in symbols:
        resp = client.commandExecute('getChartLastRequest', {'info': {"period": period, 
                                                                      "start": int(start_date_dict[period].timestamp() * 1000),
                                                                      "symbol": symbol}}, verbose=False)
        df = pd.DataFrame(resp['returnData']['rateInfos'])
        df['datetime'] = pd.to_datetime(df['ctm'], unit='ms')
        df['close'] = (df['open'] + df['close'])
        df['high'] = (df['open'] + df['high'])
        df['low'] = (df['open'] + df['low'])
        print(df.head())
        logger.info('Min. date for {} : {}'.format(symbol, df['datetime'].min()))
        df.to_csv(os.path.join(config.raw_data_path_xtb, symbol + '_' + str(period) + '.csv' ), index=False)
        del resp
        time.sleep(5)

[2022-09-25 10:33:42,138][_waitingSend:90] Sent: b'{"command": "getChartLastRequest", "arguments": {"info": {"period": 60, "start": 1546304400000, "symbol": "US500"}}}'
[2022-09-25 10:33:47,062][<module>:12] Min. date for US500 : 2020-09-09 22:00:00


             ctm                  ctmString     open    close     high  \
0  1599688800000  Sep 10, 2020, 12:00:00 AM  33975.0  34080.0  34097.0   
1  1599692400000   Sep 10, 2020, 1:00:00 AM  34080.0  34088.0  34111.0   
2  1599696000000   Sep 10, 2020, 2:00:00 AM  34088.0  33969.0  34088.0   
3  1599699600000   Sep 10, 2020, 3:00:00 AM  33969.0  33909.0  33981.0   
4  1599703200000   Sep 10, 2020, 4:00:00 AM  33913.0  33953.0  33971.0   

       low      vol            datetime  
0  33972.0  28181.0 2020-09-09 22:00:00  
1  34048.0  24084.0 2020-09-09 23:00:00  
2  33939.0  39387.0 2020-09-10 00:00:00  
3  33883.0  43800.0 2020-09-10 01:00:00  
4  33839.0  44943.0 2020-09-10 02:00:00  


[2022-09-25 10:33:52,156][_waitingSend:90] Sent: b'{"command": "getChartLastRequest", "arguments": {"info": {"period": 60, "start": 1546304400000, "symbol": "VIX"}}}'
[2022-09-25 10:33:57,492][<module>:12] Min. date for VIX : 2020-09-09 22:00:00


             ctm                  ctmString    open   close    high     low  \
0  1599688800000  Sep 10, 2020, 12:00:00 AM  2837.0  2825.0  2860.0  2822.0   
1  1599692400000   Sep 10, 2020, 1:00:00 AM  2813.0  2827.0  2833.0  2813.0   
2  1599696000000   Sep 10, 2020, 2:00:00 AM  2828.0  2854.0  2866.0  2828.0   
3  1599699600000   Sep 10, 2020, 3:00:00 AM  2861.0  2866.0  2887.0  2849.0   
4  1599703200000   Sep 10, 2020, 4:00:00 AM  2873.0  2866.0  2899.0  2866.0   

      vol            datetime  
0  3233.0 2020-09-09 22:00:00  
1  2484.0 2020-09-09 23:00:00  
2  4686.0 2020-09-10 00:00:00  
3  7169.0 2020-09-10 01:00:00  
4  4770.0 2020-09-10 02:00:00  


[2022-09-25 10:34:02,582][_waitingSend:90] Sent: b'{"command": "getChartLastRequest", "arguments": {"info": {"period": 60, "start": 1546304400000, "symbol": "US100"}}}'
[2022-09-25 10:34:08,260][<module>:12] Min. date for US100 : 2020-09-09 22:00:00


...

             ctm                  ctmString      open     close      high  \
0  1283032800000  Aug 29, 2010, 12:00:00 AM  102764.0  103156.0  106279.0   
1  1283637600000   Sep 5, 2010, 12:00:00 AM  101622.0  101302.0  103125.0   
2  1283724000000   Sep 6, 2010, 12:00:00 AM  101642.0  101094.0  101829.0   
3  1283810400000   Sep 7, 2010, 12:00:00 AM  101097.0  101109.0  101462.0   
4  1284328800000  Sep 13, 2010, 12:00:00 AM  101900.0  101905.0  102770.0   

        low  vol            datetime  
0  102194.0  0.0 2010-08-28 22:00:00  
1  100645.0  0.0 2010-09-04 22:00:00  
2  101064.0  0.0 2010-09-05 22:00:00  
3  100783.0  0.0 2010-09-06 22:00:00  
4  100603.0  0.0 2010-09-12 22:00:00  


[2022-09-25 10:39:03,252][_waitingSend:90] Sent: b'{"command": "getChartLastRequest", "arguments": {"info": {"period": 1440, "start": 1136077200000, "symbol": "USDJPY"}}}'
[2022-09-25 10:39:03,679][<module>:12] Min. date for USDJPY : 2010-08-18 22:00:00


             ctm                  ctmString     open    close     high  \
0  1282168800000  Aug 19, 2010, 12:00:00 AM  85392.0  85381.0  86642.0   
1  1282600800000  Aug 24, 2010, 12:00:00 AM  85110.0  84136.0  85186.0   
2  1282687200000  Aug 25, 2010, 12:00:00 AM  84136.0  85628.0  85810.0   
3  1282773600000  Aug 26, 2010, 12:00:00 AM  84588.0  84454.0  84869.0   
4  1282860000000  Aug 27, 2010, 12:00:00 AM  84464.0  85343.0  85442.0   

       low  vol            datetime  
0  84716.0  0.0 2010-08-18 22:00:00  
1  83578.0  0.0 2010-08-23 22:00:00  
2  84018.0  0.0 2010-08-24 22:00:00  
3  84316.0  0.0 2010-08-25 22:00:00  
4  84245.0  0.0 2010-08-26 22:00:00  


[2022-09-25 10:39:08,708][_waitingSend:90] Sent: b'{"command": "getChartLastRequest", "arguments": {"info": {"period": 1440, "start": 1136077200000, "symbol": "EURPLN"}}}'
[2022-09-25 10:39:09,274][<module>:12] Min. date for EURPLN : 2006-01-01 23:00:00


             ctm                 ctmString     open    close     high  \
0  1136156400000  Jan 2, 2006, 12:00:00 AM  38381.0  38588.0  38662.0   
1  1136242800000  Jan 3, 2006, 12:00:00 AM  38659.0  38317.0  38674.0   
2  1136329200000  Jan 4, 2006, 12:00:00 AM  38264.0  38155.0  38460.0   
3  1136415600000  Jan 5, 2006, 12:00:00 AM  38008.0  38001.0  38078.0   
4  1136502000000  Jan 6, 2006, 12:00:00 AM  38026.0  38017.0  38199.0   

       low  vol            datetime  
0  38381.0  0.0 2006-01-01 23:00:00  
1  38298.0  0.0 2006-01-02 23:00:00  
2  38111.0  0.0 2006-01-03 23:00:00  
3  37883.0  0.0 2006-01-04 23:00:00  
4  37916.0  0.0 2006-01-05 23:00:00  
